# Домашнее задание 3 [10 баллов] 
# До 30.04.18 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



## Использование архитектуры SENNA для определения части речи

Домашнее задание написано по мотивам работы R. Collobert:

**Collobert, Ronan, Jason Weston, Léon Bottou, Michael Karlen, Koray Kavukcuoglu, and Pavel Kuksa. "Natural language processing (almost) from scratch." Journal of Machine Learning Research 12, no. Aug (2011): 2493-2537.**

В этом домашнем задании вам предстоит самостоятельно разработать архитектуру SENNA для определения части речи. 
SENNA – это простая архитектура нейронной сети, позволяющая достигнуть state-of-the-art результатов в нескольких задачах обработки текстов.  

Использование SENNA для определения части речи предполагает, что задача определения части речи для данного слова формулируется как задача классификации: пусть в размеченном корпусе всего $|T|$ (= tagset) различных тегов частей речи, тогда каждое слово $w$ относится к одному из $T$ классов. Для каждого слова из обучающих данных формируется собственный вектор признаков. Нейронная сеть обучается по всем векторам признаков для слов из обучающего множества. 

Подход к решению задачи классификации представлен в оригинальной статье на рис. 1 (Figure 1: Window approach network). Он состоит из следующих шагов (раздел 3.3.1):
1. Каждое слово представляется эмбеддингом: $w_i \rightarrow LT_{w^i}$, размерность эмбеддинга - $d$;
2. Для каждого слова формируется окно длины $k$ из $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова, $k$ – нечетное. 
3. Для каждого слова формируется вектор признаков, состоящий из конкатенированных эмбеддингов слов из левого окна, данного слова и слов из правого окна. Итоговая размерность вектора признаков – $d \times k$. Именно этот вектор подается на вход нейронной сети;
4. Обучается нейронная сеть, имеющая один скрытый слой с $n_h$ нейроннами и нелинейной функцией активации $\theta$;
5. На выходном слое нейронной сети решается задача классификации на |T| классов, то есть, определяется часть речи для каждого слова. 

Если для слова невозможно найти $(k-1)/2$ соседних слов слева от данного слова  и $(k-1)/2$ соседних слов справа от данного слова – используется padding.


### Данные
1. Открытый корпус: https://github.com/dialogue-evaluation/morphoRuEval-2017/blob/master/OpenCorpora_Texts.rar
2. Предобученные эмбеддинги Facebook: https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec

### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

In [ ]:
# Читаем корпус, считаем простейшие статистики

from nltk.corpus.reader.conll import ConllCorpusReader


data_reader = ConllCorpusReader('./data', fileids='unamb_sent_14_6.conllu',
                                columntypes=['ignore', 'words', 'ignore', 'pos', 'chunk'])
sentences = list(data_reader.iob_sents())
pos_tags = [pos for sentence in sentences for word, pos, chunk in sentence]
pos_tag_set = set(pos_tags)
pos_tag_to_num = {tag: num for num, tag in enumerate(pos_tag_set)}

print(f'Количество предложений: {len(sentences)}')
print(f'Количество различных POS-тегов: {len(pos_tag_to_num)}')

Количество предложений: 38508
Количество различных POS-тегов: 14


In [2]:
# Делим предложения из корпуса на две группы: для обучающей и тестовой выборки

import random


TRAIN_PERCENT = 0.75

random.shuffle(sentences)
sentences_train_size = int(TRAIN_PERCENT * len(sentences))
sentences_train, sentences_test = sentences[:sentences_train_size], sentences[sentences_train_size:]
print(f'Количество предложений в обучающей выборке: {len(sentences_train)}')
print(f'Количество предложений в тестовой выборке: {len(sentences_test)}')

Количество предложений в обучающей выборке: 28881
Количество предложений в тестовой выборке: 9627


In [4]:
%%time
# Загружаем fasttext модель, ограничиваясь первыми limit словами

from gensim.models import KeyedVectors


fasttext_model = KeyedVectors.load_word2vec_format('wiki.ru.vec', limit=999999)

CPU times: user 24min 34s, sys: 27 s, total: 25min 1s
Wall time: 28min 14s


In [5]:
# Создаем функцию, формирующую вектор признаков для слова
# на основе его эмбеддинга и эмбеддинга его соседей

import numpy as np


def make_feature_vector(sentence, window_size, word_idx, fasttext_model):
    window_half_size = (window_size - 1) // 2
    word_vector_size = fasttext_model.vector_size
    features = np.zeros((window_size * word_vector_size,))
    idx_start = word_idx - window_half_size
    for idx in range(max(idx_start, 0), min(word_idx + window_half_size + 1, len(sentence))):
        word = sentence[idx][0].lower()
        try:
            idx_feature = (idx - idx_start) * word_vector_size
            features[idx_feature:idx_feature + word_vector_size] = fasttext_model.get_vector(word)
        except KeyError:
            # Если для слова нет ранее вычисленного эмбеддинга, то вектор его признаков оставляем нулевым
            pass
    return features

In [13]:
# Объявляем несколько полезных функций для создания генераторов данных

import itertools


def group_fixed_length(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks.
    Itertools recipe.
    Example: group('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    """
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)


def group(iterable, n):
    """Collect data into chunks or blocks. All chunks, except the last one, are of fixed length.
    Example: group('ABCDEFG', 3) --> ABC DEF G
    """
    groups = group_fixed_length(iterable, n, None)
    return (tuple(itertools.takewhile(lambda x: x is not None, gr)) for gr in groups)


def repeat_func(func, times=None, *args):
    """Repeat calls to func with specified arguments.
    Itertools recipe.
    Example:  repeatfunc(random.random)
    """
    if times is None:
        return itertools.starmap(func, itertools.repeat(args))
    return itertools.starmap(func, itertools.repeat(args, times))


def repeat_infinitely(func, *args):
    return itertools.chain.from_iterable(repeat_func(func, None, *args))

In [14]:
# Формируем обучающую и тестовую выборки.
# Т.к. при обучении модели будет использоваться categorical_crossentropy loss,
# кодируем все метки классов с помощью one-hot-encoding

from keras.utils import to_categorical


def make_word_tag_pairs(sentences, window_size, fasttext_model, pos_tag_to_num):
    words = (make_feature_vector(sentence, window_size, idx, fasttext_model)
             for sentence in sentences for idx in range(len(sentence)))
    tags = (to_categorical(pos_tag_to_num[tag], len(pos_tag_to_num))
            for sentence in sentences for _, tag, _ in sentence)
    return zip(words, tags)

    
window_size = 5
batch_size = 32
pairs_train = repeat_infinitely(make_word_tag_pairs, sentences_train, window_size,
                                fasttext_model, pos_tag_to_num)
pairs_test = repeat_infinitely(make_word_tag_pairs, sentences_test, window_size,
                               fasttext_model, pos_tag_to_num)
batches_train = group(pairs_train, batch_size)
batches_test = group(pairs_test, batch_size)

In [16]:
p = next(batches_train)
print(type(p))
print(len(p))
print(type(x))
print(x.shape)
print(y.shape)

ValueError: too many values to unpack (expected 2)

### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

In [10]:
from keras.layers import Input, Dense
from keras.models import Model

HIDDEN_LAYER_UNITS = 256


def create_classifier_model(input_shape, hidden_layer_units,
                            hidden_layer_activation, class_count):
    input_layer = Input(input_shape)
    hidden = Dense(hidden_layer_units, activation=hidden_layer_activation)(input_layer)
    output = Dense(class_count, activation='softmax')(hidden)
    return Model(inputs=[input_layer], outputs=[output])


classifier = create_classifier_model((window_size * fasttext_model.vector_size,),
                                     HIDDEN_LAYER_UNITS,
                                     'tanh',
                                     len(pos_tag_to_num))
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               384256    
_________________________________________________________________
dense_4 (Dense)              (None, 14)                3598      
Total params: 387,854
Trainable params: 387,854
Non-trainable params: 0
_________________________________________________________________


In [11]:
classifier.fit(words_train, tags_train, epochs=10, validation_data=(words_test, tags_test))

AttributeError: 'generator' object has no attribute 'ndim'

### Часть 3 [1 балл] Оценка качества

Протестируйте нейронную сеть на тестовых данных. Используйте accuracy для оценки качества модели.

### Часть 4 [1 балл] Оптимизация гиперпарметров

В эксперименте участвуют следующие гиперпараметры:
* $k$ – размер окна;
* $n_h$ – число нейронов на скрытом слое;
* $\theta$ – вид функции активации.

Оцените их влияние на качество модели. Как увеличение окна или числа нейронов влияет на итоговый показатель качества? Зависит ли итоговый показатель качества от функции активации на скрытом слое? 

### Часть 5 [2 балла] Анализ ошибок
1. Привидите примеры из тестового множества, на которых нейронная сеть ошибается. Объясните, почему возникают ошибки.
2. Протестируйте нейронную сеть на произвольном предложении (не из тестовых данных). Возникают ли ошибки? Почему?